Import libraries

In [ ]:
import pandas as pd
from snowflake.connector import connect
from datetime import datetime
from snowflake.connector.pandas_tools import write_pandas
pd.options.display.max_columns = None

## Connection

Connection to Snowflake

In [ ]:
def connect_to_sf(user, role, warehouse,schema):
    try:
        cnn = connect(
        user=user,
        authenticator='externalbrowser',
        role=role,
        warehouse=warehouse,
        database='CX_DB',
        schema=schema,
        account='cisco.us-east-1'
        )
        cs = cnn.cursor()
        return cs, cnn
    except Exception as err:
        print(err)
        print('Error connecting to Snowflake: ', "Check your credentials and VPN connection\n")
    raise
roles = {'bus': ['CX_CA_BUS_ANALYST_ROLE', 'CX_CA_RPT_WH','CX_CA_BR']}

Authentication

In [ ]:
user='user@ciscox.com'
csbus, cnnbus = connect_to_sf(user, *roles['bus'])

## QUERY

In [ ]:
## Select a sample of 20% of the distintic party IDs
sql = f'''
    with crpart as (SELECT DISTINCT BRANCH_PARTY_SSOT_PARTY_ID_INT
        FROM CX_DB.CX_CA_EBV.BV_SERVICE_REQUEST_DENORM
    WHERE UNIFORM(0.0,1.0,random())<0.2)

    SELECT *
        FROM CX_DB.CX_CA_EBV.BV_SERVICE_REQUEST_DENORM
        WHERE   
        BRANCH_PARTY_SSOT_PARTY_ID_INT IN
        (select* from crpart) AND datediff(month, INCDT_CREATION_DATE,current_date()) <=24
''' 


In [ ]:
## Query Execution
csbus.execute(sql)

In [ ]:
## Pull the Query information with the user
TAC_DATA = csbus.fetch_pandas_all() 

In [ ]:
## Download data using Parquet
TAC_DATA.to_parquet('TAC_db.parquet')

In [ ]:
## Read data
TAC_DATA = pd.read_parquet('TAC_db.parquet')